In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [16]:
# Open EletricityProductionPlant dataset
df_plants = pd.read_csv('../../Data/ElectricityProductionPlant.csv', sep=',')

# Remove all the rows that have a NaN value
df_plants = df_plants.dropna()

# Keep only the index, Canton, BeginningOfOperation, TotalPower and MainCategory columns
df_plants = df_plants[['Canton', 'BeginningOfOperation', 'TotalPower', 'MainCategory']]

df_plants.head()

,Canton,BeginningOfOperation,TotalPower,MainCategory
0,BL,2009-05-05,18.81,maincat_2
1,AG,2011-10-28,5.80,maincat_2
2,BE,2008-10-07,3.00,maincat_2
3,BE,2008-06-27,8.40,maincat_2
4,ZH,2006-04-21,4.80,maincat_2


In [17]:
# Sort by BeginningOfOperation
df_plants = df_plants.sort_values(by=['BeginningOfOperation'])
df_plants.head()

,Canton,BeginningOfOperation,TotalPower,MainCategory
162021,LU,1863-01-01,200.0,maincat_1
158324,LU,1886-01-01,740.0,maincat_1
161948,BE,1888-01-01,120.0,maincat_1
161991,SO,1888-01-01,240.0,maincat_1
158370,NW,1890-01-01,240.0,maincat_1


In [18]:
# Separate the data by canton in different dataframes and store them in a dictionary
df_plants_by_canton = {}
for canton in df_plants['Canton'].unique():
    df_plants_by_canton[canton] = df_plants[df_plants['Canton'] == canton]

#df_plants_by_canton['NW']['CumulativePower'] = df_plants_by_canton['NW']['TotalPower'].cumsum()
df_plants_by_canton['NW']['CumulativePower'] = df_plants_by_canton['NW'].groupby(['Canton', 'MainCategory'])['TotalPower'].cumsum()
df_plants_by_canton['NW'].head(15)

C:\Users\thozi\AppData\Local\Temp\ipykernel_19884\2226936664.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Canton,BeginningOfOperation,TotalPower,MainCategory,CumulativePower
158370,NW,1890-01-01,240.00,maincat_1,240.00
161895,NW,1927-04-14,18.50,maincat_1,258.50
158366,NW,1937-07-01,8500.00,maincat_1,8758.50
158367,NW,1945-11-01,6700.00,maincat_1,15458.50
158617,NW,1957-01-01,1558.00,maincat_1,17016.50
158425,NW,1960-07-01,17750.00,maincat_1,34766.50
162320,NW,1987-01-01,30.00,maincat_1,34796.50
158368,NW,1992-04-07,71.00,maincat_1,34867.50
158369,NW,1997-10-09,100.00,maincat_1,34967.50
158499,NW,1999-09-06,58.00,maincat_1,35025.50


In [19]:
#For every date, add a new row with the other main categories 
df_canton_new = {}
df_canton_new['NW'] = pd.DataFrame(columns=['Canton', 'BeginningOfOperation', 'TotalPower', 'MainCategory', 'CumulativePower'])
canton = 'NW'
cat1 = 0
cat2 = 0
cat3 = 0
cat4 = 0
for index, row in df_plants_by_canton['NW'].iterrows():
    if row['MainCategory'] == "maincat_1":
        cat1 = row['CumulativePower']
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': row['TotalPower'], 'MainCategory': row['MainCategory'], 'CumulativePower': row['CumulativePower']}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_2', 'CumulativePower': cat2}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_3', 'CumulativePower': cat3}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_4', 'CumulativePower': cat4}, ignore_index=True)
    elif row['MainCategory'] == "maincat_2":
        cat2 = row['CumulativePower']
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_1', 'CumulativePower': cat1}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': row['TotalPower'], 'MainCategory': row['MainCategory'], 'CumulativePower': row['CumulativePower']}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_3', 'CumulativePower': cat3}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_4', 'CumulativePower': cat4}, ignore_index=True)
    elif row['MainCategory'] == "maincat_3":
        cat3 = row['CumulativePower']
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_1', 'CumulativePower': cat1}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_2', 'CumulativePower': cat2}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': row['TotalPower'], 'MainCategory': row['MainCategory'], 'CumulativePower': row['CumulativePower']}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_4', 'CumulativePower': cat4}, ignore_index=True)
    elif row['MainCategory'] == "maincat_4":
        cat4 = row['CumulativePower']
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_1', 'CumulativePower': cat1}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_2', 'CumulativePower': cat2}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': 0, 'MainCategory': 'maincat_3', 'CumulativePower': cat3}, ignore_index=True)
        df_canton_new[canton] = df_canton_new[canton].append({'Canton': row['Canton'], 'BeginningOfOperation': row['BeginningOfOperation'], 'TotalPower': row['TotalPower'], 'MainCategory': row['MainCategory'], 'CumulativePower': row['CumulativePower']}, ignore_index=True)


C:\Users\thozi\AppData\Local\Temp\ipykernel_19884\240348301.py:12: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\thozi\AppData\Local\Temp\ipykernel_19884\240348301.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\thozi\AppData\Local\Temp\ipykernel_19884\240348301.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\thozi\AppData\Local\Temp\ipykernel_19884\240348301.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\thozi\AppData\Local\Temp\ipykernel_19884\240348301.py:12: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [20]:
#Display rows 60 to 100
df_canton_new['NW'].iloc[20:60]

,Canton,BeginningOfOperation,TotalPower,MainCategory,CumulativePower
20,NW,1960-07-01,17750.00,maincat_1,34766.50
21,NW,1960-07-01,0.00,maincat_2,0.00
22,NW,1960-07-01,0.00,maincat_3,0.00
23,NW,1960-07-01,0.00,maincat_4,0.00
24,NW,1987-01-01,30.00,maincat_1,34796.50
25,NW,1987-01-01,0.00,maincat_2,0.00
26,NW,1987-01-01,0.00,maincat_3,0.00
27,NW,1987-01-01,0.00,maincat_4,0.00
28,NW,1992-04-07,71.00,maincat_1,34867.50
29,NW,1992-04-07,0.00,maincat_2,0.00


In [21]:
df_canton_new['NW'].plot(kind='area',x='BeginningOfOperation', y='CumulativePower', title='NW', color='MainCategory', line_group='MainCategory')